## <span style="color:#F25C5C">SELECTOR DE SEMINARIOS</span> 

### <span style="color:#898DD9">Ten en cuenta que al menos una vez debes instalar las librerías necesarias para que funcione el script.</span>

##### <span style="color:#9BF2C1">¡En caso de tener las librerías instaladas omitir este paso!</span>

In [227]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install('pandas')
install('numpy')
install('chardet')
install('xlsxwriter')
install('openpyxl')

### <span style="color:#898DD9">Se importan las librerías necesarias para correr el script</span>

In [235]:
import pandas as pd
import numpy as np
import chardet 
import xlsxwriter
import re
import openpyxl
from openpyxl.utils import get_column_letter

### <span style="color:#898DD9">Se Analizan y cargan los archivos, revisión de la codificación para una futura exportación del archivo</span>


In [236]:
file = r"file.csv"
def detect_encoding(file_path): 
    with open(file_path, 'rb') as file: 
        detector = chardet.universaldetector.UniversalDetector() 
        for line in file: 
            detector.feed(line) 
            if detector.done: 
                break
        detector.close() 
    return detector.result['encoding'] 

encoding = detect_encoding(file) 
print(f'The encoding of the file is: {encoding}') 

The encoding of the file is: ISO-8859-1


### <span style="color:#898DD9">Carga del archivo y eliminación de las series innecesarias</span>

In [237]:
df = pd.read_csv(file, na_filter=False, sep=';', encoding= 'ISO-8859-1')
df = df.drop('Nombre', axis=1) 
df = df.drop('¿ Qué carrera profesional, técnica o tecnológica le gustaría estudiar una vez finalizada su etapa escolar?', axis=1)
df = df.drop('Correo electrónico', axis=1) 
df = df.drop('Correo electrónico2', axis=1) 
df = df.drop('ID', axis=1) 
df = df.drop('Hora de inicio', axis=1) 
df = df.drop('Hora de finalización', axis=1) 
df = df.drop('Curso', axis=1) 
df = df.drop('Colegio al cual pertenecen', axis=1) 


df.head(10)



,Nombre Completo,Ordene los seminarios desde el que mas llamo su atención al que menos atrajo su interés.
0,Linconl Javier Sánchez fajardo,INVERTEBRADOS: UN MUNDO POR DESCUBRIR;EL RASTR...
1,D?v?ra Esneider Mendivelso Caceres,INVERTEBRADOS: UN MUNDO POR DESCUBRIR;EL RASTR...
2,Nixon Sánchez,EL RASTRO DEL DELITO;AMBIENTANDO LA CIENCIA;CO...
3,Juan Sebastian Velasco Carmona,EL RASTRO DEL DELITO;COMUNICACIÓN DIGITAL;COLO...
4,Jireth Sofia Galvis Rugeles,EL RASTRO DEL DELITO;COMUNICACIÓN DIGITAL;LA B...
5,Abigail Maria Caldera Gonzalez,EL RASTRO DEL DELITO;AMBIENTANDO LA CIENCIA;CO...
6,Daniel Sebastián Muñoz Vargas,INVERTEBRADOS: UN MUNDO POR DESCUBRIR;PAP PARA...
7,María Catalina Ducuara Vela,EL RASTRO DEL DELITO;COMUNICACIÓN DIGITAL;AMBI...
8,Juan Andrés Rojas Contreras,EL RASTRO DEL DELITO;LA BRIGADA DE EMERGENCIA;...
9,Daniel Alejandro Timoteo torres,EL RASTRO DEL DELITO;LA BRIGADA DE EMERGENCIA;...


### <span style="color:#898DD9">Se toman las series necesarias y se dividen la serie por seminarios para ser analizados</span>

In [238]:
seminars = len(df['Ordene los seminarios desde el que mas llamo su atención al que menos atrajo su interés.'].iloc[0].split(';'))
list_seminars = [f"Seminario {i}" for i in range(1, seminars +1)]
df[list_seminars] = df['Ordene los seminarios desde el que mas llamo su atención al que menos atrajo su interés.'].str.split(';', n=seminars-1, expand=True)
df = df.drop(columns=[list_seminars[-1]])
df = df.drop('Ordene los seminarios desde el que mas llamo su atención al que menos atrajo su interés.', axis=1) 
df.head()

,Nombre Completo,Seminario 1,Seminario 2,Seminario 3,Seminario 4,Seminario 5,Seminario 6,Seminario 7
0,Linconl Javier Sánchez fajardo,INVERTEBRADOS: UN MUNDO POR DESCUBRIR,EL RASTRO DEL DELITO,COLORES DE RESILIENCIA,COMUNICACIÓN DIGITAL,AMBIENTANDO LA CIENCIA,LA BRIGADA DE EMERGENCIA,PAP PARA LA VIDA
1,D?v?ra Esneider Mendivelso Caceres,INVERTEBRADOS: UN MUNDO POR DESCUBRIR,EL RASTRO DEL DELITO,AMBIENTANDO LA CIENCIA,COLORES DE RESILIENCIA,LA BRIGADA DE EMERGENCIA,COMUNICACIÓN DIGITAL,PAP PARA LA VIDA
2,Nixon Sánchez,EL RASTRO DEL DELITO,AMBIENTANDO LA CIENCIA,COLORES DE RESILIENCIA,PAP PARA LA VIDA,COMUNICACIÓN DIGITAL,INVERTEBRADOS: UN MUNDO POR DESCUBRIR,LA BRIGADA DE EMERGENCIA
3,Juan Sebastian Velasco Carmona,EL RASTRO DEL DELITO,COMUNICACIÓN DIGITAL,COLORES DE RESILIENCIA,AMBIENTANDO LA CIENCIA,PAP PARA LA VIDA,LA BRIGADA DE EMERGENCIA,INVERTEBRADOS: UN MUNDO POR DESCUBRIR
4,Jireth Sofia Galvis Rugeles,EL RASTRO DEL DELITO,COMUNICACIÓN DIGITAL,LA BRIGADA DE EMERGENCIA,PAP PARA LA VIDA,COLORES DE RESILIENCIA,INVERTEBRADOS: UN MUNDO POR DESCUBRIR,AMBIENTANDO LA CIENCIA


### <span style="color:#898DD9">Se crea un diccionario para organizar los estudiantes y la preferencia de los seminarios </span>

In [241]:
student_preferences = {}
for index, row in df.iterrows():
    nombre_completo = row['Nombre Completo']
    seminar_preferences = [v for v in row[1:] if v]
    student_preferences[nombre_completo] = seminar_preferences
    
print (student_preferences)

{'Linconl Javier Sánchez fajardo ': ['INVERTEBRADOS: UN MUNDO POR DESCUBRIR', 'EL RASTRO DEL DELITO', 'COLORES DE RESILIENCIA', 'COMUNICACIÓN DIGITAL', 'AMBIENTANDO LA CIENCIA', 'LA BRIGADA DE EMERGENCIA', 'PAP PARA LA VIDA'], 'D?v?ra Esneider Mendivelso Caceres': ['INVERTEBRADOS: UN MUNDO POR DESCUBRIR', 'EL RASTRO DEL DELITO', 'AMBIENTANDO LA CIENCIA', 'COLORES DE RESILIENCIA', 'LA BRIGADA DE EMERGENCIA', 'COMUNICACIÓN DIGITAL', 'PAP PARA LA VIDA'], 'Nixon Sánchez ': ['EL RASTRO DEL DELITO', 'AMBIENTANDO LA CIENCIA', 'COLORES DE RESILIENCIA', 'PAP PARA LA VIDA', 'COMUNICACIÓN DIGITAL', 'INVERTEBRADOS: UN MUNDO POR DESCUBRIR', 'LA BRIGADA DE EMERGENCIA'], 'Juan Sebastian Velasco Carmona ': ['EL RASTRO DEL DELITO', 'COMUNICACIÓN DIGITAL', 'COLORES DE RESILIENCIA', 'AMBIENTANDO LA CIENCIA', 'PAP PARA LA VIDA', 'LA BRIGADA DE EMERGENCIA', 'INVERTEBRADOS: UN MUNDO POR DESCUBRIR'], 'Jireth Sofia Galvis Rugeles': ['EL RASTRO DEL DELITO', 'COMUNICACIÓN DIGITAL', 'LA BRIGADA DE EMERGENCIA', '

### <span style="color:#898DD9">Se definen las variables que nos determinan el total de estudiantes, el total de seminarios y cuantos estudiantes hay por seminario.    </span>

In [243]:
total_seminars = seminars - 1
print(f'Total Seminarios: {total_seminars}')

total_students = len(df.index)
print(f'\nNumero de estudiantes: {total_students}')

students_seminar =  np.ceil(total_students/total_seminars)
print(f'\nNumero de estudiantes por seminario: {students_seminar}')

Total Seminarios: 7

Numero de estudiantes: 97

Numero de estudiantes por seminario: 14.0


### <span style="color:#898DD9">Se crea un diccionario para organizar los seminarios y la cantidad máxima de estudiantes por seminario </span>

In [244]:
seminar_capacities = {}
for student, preferences in student_preferences.items():
    for seminar in preferences:
        if seminar not in seminar_capacities:
            seminar_capacities[seminar] = students_seminar
            
print (seminar_capacities)

{'INVERTEBRADOS: UN MUNDO POR DESCUBRIR': 14.0, 'EL RASTRO DEL DELITO': 14.0, 'COLORES DE RESILIENCIA': 14.0, 'COMUNICACIÓN DIGITAL': 14.0, 'AMBIENTANDO LA CIENCIA': 14.0, 'LA BRIGADA DE EMERGENCIA': 14.0, 'PAP PARA LA VIDA': 14.0}


### <span style="color:#898DD9">Se asigna el seminario a cada estudiante en base al orden de inscripción y los cupos disponibles </span>

In [245]:
def assign_seminars(student_preferences, seminar_capacities):
    assigned_seminars = {}
    for student, preferences in student_preferences.items():
        for seminar in preferences:
            if seminar_capacities[seminar] > 0:
                assigned_seminars[student] = seminar
                seminar_capacities[seminar] -= 1
                break
    return assigned_seminars

assigned_seminars = assign_seminars(student_preferences, seminar_capacities)
print(assigned_seminars)

{'Linconl Javier Sánchez fajardo ': 'INVERTEBRADOS: UN MUNDO POR DESCUBRIR', 'D?v?ra Esneider Mendivelso Caceres': 'INVERTEBRADOS: UN MUNDO POR DESCUBRIR', 'Nixon Sánchez ': 'EL RASTRO DEL DELITO', 'Juan Sebastian Velasco Carmona ': 'EL RASTRO DEL DELITO', 'Jireth Sofia Galvis Rugeles': 'EL RASTRO DEL DELITO', 'Abigail Maria Caldera Gonzalez': 'EL RASTRO DEL DELITO', 'Daniel Sebastián Muñoz Vargas ': 'INVERTEBRADOS: UN MUNDO POR DESCUBRIR', 'María Catalina Ducuara Vela': 'EL RASTRO DEL DELITO', 'Juan Andrés Rojas Contreras ': 'EL RASTRO DEL DELITO', 'Daniel Alejandro Timoteo torres ': 'EL RASTRO DEL DELITO', 'Edward Santiago Torres Ariza ': 'EL RASTRO DEL DELITO', 'Zara Jireht Grisales Pérez ': 'EL RASTRO DEL DELITO', 'María Fernanda Vega Lugo ': 'EL RASTRO DEL DELITO', 'David santiago tapiero figueroa ': 'INVERTEBRADOS: UN MUNDO POR DESCUBRIR', 'Alisson Reyes Ibarra': 'EL RASTRO DEL DELITO', 'Michel Contreras Rendón': 'EL RASTRO DEL DELITO', 'Ana María Rodríguez Tafur ': 'INVERTEBRADO

### <span style="color:#898DD9">El diccionario se convierte a un dataframe para ordenarlo antes de exportarlo a un archico de excel </span>

In [ ]:
final_df = pd.DataFrame(list(assigned_seminars.items()), columns=['Nombre', 'Seminario'])

final_df.head()

### <span style="color:#898DD9">Se exporta el dataframe en un archivo de Excel, creando una hoja por seminario y se ordenan los estudiantes por seminario </span>


In [225]:
workbook = xlsxwriter.Workbook('asignacion_seminarios.xlsx')
seminarios = final_df['Seminario'].unique()

for seminario in seminarios:
    sheet_name = re.sub(r'[:?/\\*\[\]\'"]', '_', seminario)[:31]
    worksheet = workbook.add_worksheet(sheet_name)
    worksheet.set_default_row(15)
    seminario_df = final_df[final_df['Seminario'] == seminario].dropna(how='all')
    worksheet.write_row(0, 0, seminario_df.columns.tolist())
    for index, row in seminario_df.iterrows():
        worksheet.write_row(index + 1, 0, row.tolist())

workbook.close()

### <span style="color:#898DD9">Se ordena el archivo eliminando espacios vacios y se organiza el archivo </span>

In [226]:

wb = openpyxl.load_workbook('asignacion_seminarios.xlsx')
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    for row in range(ws.max_row, 0, -1):
        if all(cell.value is None for cell in ws[row]):
            ws.delete_rows(row)


wb.save('asignacion_seminarios.xlsx')